In [ ]:
%matplotlib notebook


import numpy as np
import pandas as pd
import dask
import os

from operator import itemgetter

import numpy as np
import scipy
import matplotlib.pyplot as plt

import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from dask_ml.linear_model import LogisticRegression
from dask_ml.model_selection import train_test_split
from dask_ml.datasets import make_classification
from dask.diagnostics import ProgressBar
from dask_ml.model_selection import IncrementalSearchCV
import dask_ml.datasets
import dask_ml.cluster


import matplotlib.pyplot as plt
import numpy as np
from mlxtend.plotting import plot_decision_regions

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

import joblib


In [ ]:
# Handle and preprocess data
X = feature_array
y = target_array.flatten()

# Split data into train and validation
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Scale data
sc = StandardScaler()
sc.fit(X_train)

# Transform (standardise) both X_train and X_test with mean and STD from
# training data
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)



## Combining transformers and estimators in a pipeline

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        LogisticRegression(solver = 'lbfgs', multi_class='auto', random_state=1))
pipe_lr

In [ ]:
pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
print('Test Accuracy: %.3f' % pipe_lr.score(X_test, y_test))

## Simple logistic regression model

In [ ]:
# Train the model using grid search optimization
parameters = {'penalty': ['l1', 'l2'], 'C': [0.5, 1, 2]}

lr = LogisticRegression()
tuned_lr = GridSearchCV(lr, parameters)

with ProgressBar():
    tuned_lr.fit(X_train, y_train)

In [ ]:
# Score the model
lr.score(X_test, y_test).compute()

In [ ]:
# Visualizing the score
pd.DataFrame(tuned_lr.cv_results_)

In [ ]:
# Get hyperparameters of best performing model
tuned_lr.best_params_

In [ ]:
# Fit the best classifier
clf = tuned_lr.best_params_
with ProgressBar():
    clf.fit(X_train, y_train)
    
print('Test accuracy: %.3f' % clf.score(X_test, y_test))

## Compute the area under curve (AUC), F1 and confusion matrix

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score, recall_score, f1_score

print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred))
print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=y_pred))



## Plot the Receiver Operating Characteristic (ROC) for your best model on the training data

In [ ]:
from sklearn.metrics import roc_curve, auc
from scipy import interp

pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        LogisticRegression(solver = 'lbfgs', max_iter = 200, 
                                           penalty = 'l2', multi_class = 'auto',  
                                           random_state=1, C=100.0))
X_train2 = X_train[:, [4, 14]]
    
cv = list(StratifiedKFold(n_splits=3, 
                          random_state=1).split(X_train, y_train))

fig = plt.figure(figsize=(7, 5))

mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
all_tpr = []

for i, (train, test) in enumerate(cv):
    probas = pipe_lr.fit(X_train2[train],
                         y_train[train]).predict_proba(X_train2[test])

    [fpr, tpr, thresholds] = roc_curve(y_train[test],
                                     probas[:, 1],
                                     pos_label=1)
    mean_tpr += interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr,
             tpr,
             label='ROC fold %d (area = %0.2f)'
                   % (i+1, roc_auc))

plt.plot([0, 1],
         [0, 1],
         linestyle='--',
         color=(0.6, 0.6, 0.6),
         label='random guessing')

mean_tpr    /= len(cv)
mean_tpr[-1] = 1.0
mean_auc     = auc(mean_fpr, mean_tpr)

plt.plot(mean_fpr, mean_tpr, 'k--',
         label='mean ROC (area = %0.2f)' % mean_auc, lw=2)
plt.plot([0, 0, 1],
         [0, 1, 1],
         linestyle=':',
         color='black',
         label='perfect performance')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.legend(loc="lower right")

plt.tight_layout()
# plt.savefig('images/06_10.png', dpi=300)
plt.show()

### What are the dimensions of your training and test data? 
### Why do we use F1 (Dice) as the scoring metric in Kaggle and not accuracy? 